In [1]:
include("src\\ImportanceSampling.jl")
using ImportanceSampling
using Distributions
using Seaborn

In [7]:
p = MvNormal(2.0*ones(2), eye(2))
q = MvTDist(1, zeros(2), eye(2)*5)
f!(r, x) = r .= vcat(x, x.^2)

is = ImportanceSampler(f!, 4, q, p=p)
update!(is, batchsize=1000, nbatches=10)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] sqrt(::Array{Float64,2}) at .\deprecated.jl:57
 [3] _rand!(::Distributions.GenericMvTDist{Float64,PDMats.PDMat{Float64,Array{Float64,2}}}, ::Array{Float64,2}) at C:\Users\Hamza El-Saawy\.julia\v0.6\Distributions\src\multivariate\mvtdist.jl:176
 [4] rand!(::Distributions.GenericMvTDist{Float64,PDMats.PDMat{Float64,Array{Float64,2}}}, ::Array{Float64,2}) at C:\Users\Hamza El-Saawy\.julia\v0.6\Distributions\src\multivariates.jl:36
 [5] #_update!#27(::Bool, ::Array{Any,1}, ::Function, ::Val{true}, ::ImportanceSampling.ImportanceSampler, ::Void, ::Void, ::Void, ::Int64, ::Int64, ::Int64) at C:\Users\Hamza El-Saawy\Documents\school\stats362\project\src\importance_sampler.jl:189
 [6] #update!#13(::Void, ::Void, ::Void, ::Int64, ::Int64, ::Int64, ::Array{Any,1}, ::Function, ::ImportanceSampling.ImportanceSampler) at C:\Users\Hamza El-Saawy\Documents\school\stats362\project\src\importance_sampler.jl:63
 [7] (::ImportanceSam

ImportanceSampling.ImportanceSampler(Distributions.GenericMvTDist{Float64,PDMats.PDMat{Float64,Array{Float64,2}}}(
df: 1.0
dim: 2
zeromean: true
μ: [0.0, 0.0]
Σ: PDMats.PDMat{Float64,Array{Float64,2}}(2, [5.0 0.0; 0.0 5.0], Base.LinAlg.Cholesky{Float64,Array{Float64,2}} with factor:
[2.23607 0.0; 0.0 2.23607])
)
, ImportanceSampling.MeanVariance{4}(10000, [20555.1, 21098.4, 51333.2, 52919.0], [6.30564e5 5.70084e5 1.6703e6 1.39834e6; 5.70084e5 6.48311e5 1.38438e6 1.71692e6; 1.6703e6 1.38438e6 4.71977e6 3.38835e6; 1.39834e6 1.71692e6 3.38835e6 4.83106e6]), ImportanceSampling.Diagnostic(10000, 10359.559976216706, 125704.62861348175, 2.0922847037500395e6, 3.966254111856501e7), ImportanceSampling.#22, f!)

In [8]:
m = mean(is)
m[1:2]

2-element Array{Float64,1}:
 2.05551
 2.10984

In [11]:
m[3:4] .- m[1:2].^2

2-element Array{Float64,1}:
 0.908199
 0.840454

In [14]:
var(is)

4-element Array{Float64,1}:
 0.00630627
 0.00648376
 0.0472024 
 0.0483154 